# Подготовка данных

## Импорт данных

In [1]:
import pandas as pd
dataset = pd.read_csv('../data/machine_preprocessed.csv')
dataset.head()

,myct,mmin,mmax,cach,chmin,chmax,prp,erp,vendor_adviser,vendor_amdahl,...,vendor_microdata,vendor_nas,vendor_ncr,vendor_nixdorf,vendor_perkin-elmer,vendor_prime,vendor_siemens,vendor_sperry,vendor_sratus,vendor_wang
0,125,256,6000,256,16,128,198,199,1,0,...,0,0,0,0,0,0,0,0,0,0
1,29,8000,32000,32,8,32,269,253,0,1,...,0,0,0,0,0,0,0,0,0,0
2,29,8000,32000,32,8,32,220,253,0,1,...,0,0,0,0,0,0,0,0,0,0
3,29,8000,32000,32,8,32,172,253,0,1,...,0,0,0,0,0,0,0,0,0,0
4,29,8000,16000,32,8,16,132,132,0,1,...,0,0,0,0,0,0,0,0,0,0


## Аномалии

Исходя из графиков визуализации данных, выполненных в файле 1_visualization можно предположить, что в данных присутствуют аномалии, однако данный датасет составлялся не на основе сбора данных людей, а на основе конкретных технических характеристик процессоров, в связи с этим нельзя отбросить какие-либо данные

## Разбиение данных

Для начала разобьем данные на множество описаний объектов и множество меток:

In [2]:
X = dataset.drop('erp', 1).values
y = dataset['erp'].values

Разобьем на обучающую и тестовую выборки:

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)

# Регрессия

Функция для вывода значений оценки

In [38]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def predict_estimation(estimator, y_test, y_pred):
    print("R^2 = ", estimator.score(X, y))
    print("MSE = ", mean_squared_error(y_test, y_pred))
    print("RMSE = ", mean_squared_error(y_test, y_pred, squared = False))
    print("MAE = ", mean_absolute_error(y_test, y_pred))

## Linear Regression

Документация:

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

Обучение модели:

In [39]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

Оценим качество модели:

In [40]:
predict_estimation(lr, y_test, lr.predict(X_test))

R^2 =  0.9579022762171572
MSE =  2842.4438642431314
RMSE =  53.31457459497479
MAE =  24.364680859035552


Выведем полученные и предполагаемые значения:

In [57]:
print(lr.predict(X_test))
print(y_test)

[166.4020012   31.768259    11.55547183 782.46187472  33.51170553
 300.58452334  43.033258    -5.66443436 946.34848287 236.10516337
  95.83582609 128.58227406  39.86281274  24.40447689  87.47569573
   8.78753014  62.45653232   4.3598919  155.42621044 101.30109955
  54.81084671  54.58844002 120.08776487  27.63989149   8.64004026
  27.47850058  30.99645211  -4.41609568   2.7576834   87.91506789
 270.55222691  31.01185511  21.38929372   9.3809198    3.40627302
  46.05163593  66.00122283 387.59753727  85.78980166  16.53707066
 297.35705539  50.6796723 ]
[ 102   25   25  919   34  267   41   19 1238  227   88   99   41   44
   64   23   80   22  142   82   52   56  119   26   17   41   21   30
   23   67  275   33   35   20   20   46   47  361   73   26  341   47]


Для данного метода отсутсвуют гиперпараметры

## Decision Tree Regressor

Документация:

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

Обучение модели:

In [55]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)

DecisionTreeRegressor()

Оценим качество модели:

In [58]:
predict_estimation(dtr, y_test, dtr.predict(X_test))

R^2 =  0.9432817120140219
MSE =  6727.261904761905
RMSE =  82.01988725158006
MAE =  25.071428571428573


Выведем полученные и предполагаемые значения:

In [59]:
print(dtr.predict(X_test))
print(y_test)

[102.  24.  28. 978.  34. 270.  38.  18. 749. 181.  88. 117.  39.  44.
  64.  24.  56.  21. 117. 117.  50.  50. 117.  28.  18.  44.  20.  28.
  22.  50. 183.  31.  41.  20.  20.  46.  53. 381.  64.  28. 181.  39.]
[ 102   25   25  919   34  267   41   19 1238  227   88   99   41   44
   64   23   80   22  142   82   52   56  119   26   17   41   21   30
   23   67  275   33   35   20   20   46   47  361   73   26  341   47]


Найдем наилучие гиперпараметры:

In [63]:
from sklearn.model_selection import GridSearchCV
param_grid = {"criterion": ["mse", "mae"],
              "min_samples_split": [10, 20, 40],
              "max_depth": [2, 6, 8],
              "min_samples_leaf": [20, 40, 100],
              "max_leaf_nodes": [5, 20, 100],
              }
grid_cv_dtr = GridSearchCV(dtr, param_grid, cv=5).fit(X,y)
print("Best Hyperparameters:\n{}".format(grid_cv_dtr.best_params_))

Best Hyperparameters:
{'criterion': 'mae', 'max_depth': 6, 'max_leaf_nodes': 20, 'min_samples_leaf': 20, 'min_samples_split': 10}


## Lasso

Документация:

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso

Обучение модели:

In [43]:
from sklearn.linear_model import Lasso

lo = Lasso()
lo.fit(X_train, y_train)

Lasso()

Оценим качество модели:

In [44]:
predict_estimation(lo, y_test, lo.predict(X_test))

R^2 =  0.9520313863482137
MSE =  2984.9146055903616
RMSE =  54.634372016070266
MAE =  25.428425431961625


Выведем полученные и предполагаемые значения:

In [54]:
print(lo.predict(X_test))
print(y_test)

[ 1.62145206e+02  1.15864671e+01  1.20909336e+01  7.68366620e+02
  3.81826708e+01  3.11170198e+02  5.00216135e+01 -1.02450972e+01
  9.44780178e+02  2.33398592e+02  8.53966285e+01  1.25923046e+02
  4.16445289e+01  3.64465288e+01  6.67437367e+01  1.62515637e+01
  5.72160960e+01  3.91036977e+00  1.35267854e+02  1.15814531e+02
  6.15784976e+01  6.02826123e+01  1.26425262e+02  2.87946414e+01
  1.01324058e+01  3.93155203e+01  2.52448786e+01  8.96813783e+00
 -2.41511980e-01  5.76047902e+01  2.71005320e+02  1.86026673e+01
  2.70066657e+01 -3.40314827e+00  7.47110935e+00  3.06662756e+01
  6.25414063e+01  4.04145841e+02  9.39012537e+01  1.09655320e+00
  3.07648694e+02  3.70230259e+01]
[ 102   25   25  919   34  267   41   19 1238  227   88   99   41   44
   64   23   80   22  142   82   52   56  119   26   17   41   21   30
   23   67  275   33   35   20   20   46   47  361   73   26  341   47]


Найдем наилучие гиперпараметры:

In [73]:
param_grid = {'alpha':[0.02, 0.024, 0.025, 0.026, 0.03]}
grid_cv_lo = GridSearchCV(lo, param_grid).fit(X,y)
print("Best Hyperparameters:\n{}".format(grid_cv_lo.best_params_))

Best Hyperparameters:
{'alpha': 0.03}


## Ridge

Документация:

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge

Обучение модели:

In [47]:
from sklearn.linear_model import Ridge

rg = Ridge()
rg.fit(X_train, y_train)

Ridge()

Оценим качество модели:

In [48]:
predict_estimation(rg, y_test, rg.predict(X_test))

R^2 =  0.9583257786465182
MSE =  2752.702496109876
RMSE =  52.46620337045435
MAE =  23.543692073153185


Выведем полученные и предполагаемые значения:

In [52]:
print(rg.predict(X_test))
print(y_test)

[ 1.61869096e+02  2.23095854e+01  1.24742743e+01  7.79666931e+02
  3.55498454e+01  3.01961215e+02  4.46876249e+01 -6.81405613e+00
  9.51414530e+02  2.37127306e+02  9.34803635e+01  1.27626772e+02
  4.07952338e+01  2.77201314e+01  7.58519148e+01  1.06280869e+01
  6.41295605e+01  4.54212687e+00  1.53145774e+02  1.03634798e+02
  5.60335386e+01  5.53850039e+01  1.19533352e+02  2.70389823e+01
  8.21423539e+00  2.94699804e+01  3.06398459e+01 -4.73392619e-01
  1.96787488e+00  7.43734298e+01  2.70389067e+02  2.86561972e+01
  2.04526063e+01  6.95792183e+00  3.28379210e+00  4.24793938e+01
  6.69426790e+01  3.92014692e+02  8.64510262e+01  1.57363203e+01
  3.00561670e+02  5.08198841e+01]
[ 102   25   25  919   34  267   41   19 1238  227   88   99   41   44
   64   23   80   22  142   82   52   56  119   26   17   41   21   30
   23   67  275   33   35   20   20   46   47  361   73   26  341   47]


Найдем наилучие гиперпараметры:

In [74]:
param_grid = {'alpha':[200, 230, 250, 265, 270, 275, 290, 300, 500]}
grid_cv_rg = GridSearchCV(rg, param_grid).fit(X,y)
print("Best Hyperparameters:\n{}".format(grid_cv_rg.best_params_))

Best Hyperparameters:
{'alpha': 500}


## ElasticNet

Документация:

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet

Обучение модели:

In [50]:
from sklearn.linear_model import ElasticNet

en = ElasticNet()
en.fit(X_train, y_train)

ElasticNet()

Оценим качество модели:

In [51]:
predict_estimation(en, y_test, en.predict(X_test))

R^2 =  0.9520523651064292
MSE =  2982.9332858583343
RMSE =  54.61623646735771
MAE =  25.3730408163837


Выведем полученные и предполагаемые значения:

In [53]:
print(en.predict(X_test))
print(y_test)

[ 1.62481347e+02  1.16918531e+01  1.21447044e+01  7.68216772e+02
  3.82836752e+01  3.10896082e+02  5.01857522e+01 -1.01601081e+01
  9.45022950e+02  2.31927964e+02  8.54978200e+01  1.26068160e+02
  4.17487628e+01  3.65222450e+01  6.68971221e+01  1.63031181e+01
  5.77781618e+01  3.95678613e+00  1.35959606e+02  1.15799742e+02
  6.16837326e+01  6.04716000e+01  1.25996856e+02  2.87451972e+01
  1.00818014e+01  3.88801672e+01  2.54190539e+01  8.99859138e+00
 -1.70076590e-01  5.77441760e+01  2.71084195e+02  1.86600028e+01
  2.70776400e+01 -3.30072761e+00  7.43315367e+00  3.07859717e+01
  6.27591315e+01  4.04024243e+02  9.41739085e+01  1.94110825e+00
  3.05970200e+02  3.77745035e+01]
[ 102   25   25  919   34  267   41   19 1238  227   88   99   41   44
   64   23   80   22  142   82   52   56  119   26   17   41   21   30
   23   67  275   33   35   20   20   46   47  361   73   26  341   47]


Найдем наилучие гиперпараметры:

In [78]:
param_grid = {'alpha': np.arange(0.1,2,0.05), 'l1_ratio': np.arange(0.1, 1.1, 0.1)}
grid_cv_en = GridSearchCV(en, param_grid).fit(X,y)
print("Best Hyperparameters:\n{}".format(grid_cv_en.best_params_))

Best Hyperparameters:
{'alpha': 1.9500000000000006, 'l1_ratio': 1.0}


## Вывод

Максимальный коэффициент детерминации имеет Ridge, следовательно является лучшим для данной модели